Let's play tictactoe!

In [1]:
import random

def line_has(a,b,c,x):
    return a == x or b == x or c == x

def check_win(board):
    might_win = False
    for x in range(3):
        if board[x][0] != ' ' and board[x][0] == board[x][1] == board[x][2]:
            return board[x][0]
        if not might_win and not line_has(board[x][0], board[x][1], board[x][2], 'x') or not line_has(board[x][0], board[x][1], board[x][2], 'o'):
            might_win = True
    for x in range(3):
        if board[0][x] != ' ' and board[0][x] == board[1][x] == board[2][x]:
            return board[x][0]
        if not might_win and not line_has(board[0][x], board[1][x], board[2][x], 'x') or not line_has(board[0][x], board[1][x], board[2][x], 'o'):
            might_win = True
    if board[0][0] != ' ' and board[0][0] == board[1][1] == board[2][2]:
        return board[0][0]
    if not might_win and not line_has(board[0][0], board[1][1], board[2][2], 'x') or not line_has(board[0][0], board[1][1], board[2][2], 'o'):
        might_win = True
    if board[0][2] != ' ' and board[0][2] == board[1][1] == board[2][0]:
        return board[0][2]
    if not might_win and not line_has(board[0][2], board[1][1], board[2][0], 'x') or not line_has(board[0][2], board[1][1], board[2][0], 'o'):
        might_win = True
    if not might_win:
        return ' '
    return ''

def display_board(b):
    print(f" {b[0][0]} | {b[0][1]} | {b[0][2]} ")
    print("---+---+---")
    print(f" {b[1][0]} | {b[1][1]} | {b[1][2]} ")
    print("---+---+---")
    print(f" {b[2][0]} | {b[2][1]} | {b[2][2]} ")
    print()

def play_random(board):
    while True:
        x = random.randint(0,2)
        y = random.randint(0,2)
        if board[x][y] == ' ':
            return (x,y)

def play_human(board):
    move = input("What is your next move? Type eg \"1,2\"")
    x = int(move[0])
    y = int(move[2])
    return x,y

def play_human_suggestor(table):
    def a(board):
        print("What is your next move?")
        suggestor = play_table(table)
        print(f"I suggest {suggestor(board)}")
        move = input("Type eg \"1,2\": ")
        x = int(move[0])
        y = int(move[2])
        return x,y
    return a

should_display=False
def play_table(table):
    def a(board):
        key = str(board)
        if key in table:
            state = table[key]
            best = max(state, key=state.get)
            best_score = state[best]
            if should_display:
                print(f"table state is {key} => {state}")
                print(f"best move = {best}, with a score of {best_score}")
            if best_score <= 0:
                return play_random(board)
            else:
                if should_display:
                    print(f"playing best move {best}")
                return best
        else:
            return play_random(board)
    return a

def find_unscored_candidates(board, state):
    candidates = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == ' ' and not ((i,j) in state):
                candidates.append((i,j))
            # else:
            #     print(f"candidate {(i,j)} rejected")
            #     print(f"board free {board[i][j] == ' '}")
            #     print(f"not scored {not ((i,j) in state)}")
    return candidates
    
def play_trainer(table, n, threshold):
    def a(board):
        key = str(board)
        if key in table:
            state = table[key]
            best = max(state, key=state.get)
            best_score = state[best]
            if best_score <= 0:
                # when best score is less than zero, randomly pick a spot that is not filled or scored
                # print("\n\nBest score is less than zero!")
                # print(f"board = {board}")
                # print(f"state = {state}")
                candidates = find_unscored_candidates(board, state)
                # print(f"candidates found = {candidates}")
                if len(candidates) > 0:
                    res = random.choice(candidates)
                    # print(f"returning {res}")
                    return res
                else:
                    # print(f"returning random")
                    return play_random(board)
            elif best_score >= threshold:
                return best
            else:
                if random.random() < best_score / threshold: # and random.random() < n:
                    return best
                else:
                    return play_random(board)
        else:
            return play_random(board)
    return a

def new_board():
    return [[' ',' ',' '],[' ',' ',' '],[' ',' ',' ']]


def play_game(agent_x, agent_o, should_display=False):
    board = new_board()
    turn = 'x'
    sequence = []
    for count in range(9):
        # play a move
        if turn == 'x':
            x, y = agent_x(board)
        else:
            x, y = agent_o(board)
        board[x][y] = turn
        sequence.append((x,y))
        # print board
        if should_display:
            display_board(board)
        # check if someone won
        if check_win(board) != '':
            break
        # new turn
        if turn == 'x':
            turn = 'o'
        else:
            turn = 'x'
    return check_win(board), sequence

def did_i_win(table, board):
    key = str(board)
    if key in table:
        state = table[key]
        best = max(state, key=state.get)
        best_score = state[best]
        if best_score >= 1000:
            print(f"\nI just won!")
            print(f"board={board} best={best}")
            print(f"state={state}")
            return True
    return False

def play_game_fast_with_table(agent_x, agent_o, table, should_display=False):
    board = new_board()
    turn = 'x'
    sequence = []
    for count in range(9):
        # play a move
        if turn == 'x':
            x, y = agent_x(board)
        else:
            x, y = agent_o(board)
        board[x][y] = turn
        # print board
        if should_display:
            display_board(board)
        # check premature win
        if did_i_win(table, board):
            print(f"winner is {turn}")
            winner = turn
            break
        sequence.append((x,y))
        # check if someone won
        winner = check_win(board)
        if winner != '':
            break
        # new turn
        if turn == 'x':
            turn = 'o'
        else:
            turn = 'x'
    return winner, sequence

def train(n):
    table = {}
    for i in range(n):
        winner, sequence = play_game_fast_with_table(play_trainer(table, i/(n*1.5), 800), play_trainer(table, i/(n*1.5), 800), table)
        # reverse the sequence, then award a score based on who won. If draw, no weight change.
        if winner == ' ':
            continue
        board = new_board()
        turn = 'x'
        for i in range(len(sequence)):
            key = str(board)
            #print(f"board is {key}")
            if key not in table:
                table[key] = {}
            move = (x,y) = sequence[i]
            #print(f"move is {move}")
            if move not in table[key]:
                table[key][move] = 0
            score = int(1000 / 2 ** ((len(sequence) - i - 1)/2))
            #print(f"score is {score}")
            if turn == winner:  # this player won
                #print("this is a winning play")
                table[key][move] = int(table[key][move] / 2) + score
            else:
                #print("this is a losing play")
                table[key][move] = int(table[key][move] / 2) - score
            board[x][y] = turn
            # new turn
            if turn == 'x':
                turn = 'o'
            else:
                turn = 'x'
    return table

def test(n, agent_x, agent_o):
    o_wins = x_wins = draws = 0
    for i in range(n):
        winner, sequence = play_game(agent_x, agent_o)
        if winner == 'x':
            x_wins += 1
        elif winner == 'o':
            o_wins += 1
        else:
            draws += 1
    print(f"x wins {int(x_wins*100/n)}%")
    print(f"o wins {int(o_wins*100/n)}%")
    print(f"draws {int(draws*100/n)}%")

table_small = train(1000)
# table_big = train(10000)
# #print(table)
# print("Round 1: Random X vs Random O")
# test(10000, play_random, play_random)
# print()
# print("Round 2: Random X vs Small Table O")
# test(10000, play_random, play_table(table_small))
# print()
# print("Round 2: Random X vs Big Table O")
# test(10000, play_random, play_table(table_big))
# print()

# should_display = True
# winner, sequence = play_game(play_table(table_big), play_human_suggestor(table_big), True)
# if winner != ' ':
#     print(f"{winner} wins!")
# else:
#     print(f"draw!")
# print(sequence)



I just won!
board=[['o', 'x', 'x'], ['x', 'o', 'x'], ['o', ' ', ' ']] best=(2, 2)
state={(2, 2): 1000}
winner is x

I just won!
board=[['o', 'x', 'x'], ['x', 'x', 'o'], ['o', ' ', 'o']] best=(2, 1)
state={(2, 1): 1000}
winner is o

I just won!
board=[[' ', 'x', 'o'], ['o', 'o', 'x'], [' ', 'x', 'x']] best=(2, 0)
state={(2, 0): 1000}
winner is x

I just won!
board=[['x', ' ', 'o'], ['x', ' ', 'x'], [' ', 'o', 'o']] best=(1, 1)
state={(0, 1): -707, (1, 1): 1000}
winner is o

I just won!
board=[['x', ' ', 'x'], [' ', 'x', ' '], [' ', 'o', 'o']] best=(2, 0)
state={(2, 0): 1000}
winner is x

I just won!
board=[[' ', 'x', 'x'], [' ', 'o', 'o'], ['o', 'x', 'x']] best=(1, 0)
state={(1, 0): 1000}
winner is x

I just won!
board=[['o', 'o', 'x'], ['x', ' ', 'x'], ['x', 'o', 'o']] best=(1, 1)
state={(1, 1): 1000}
winner is o

I just won!
board=[['o', ' ', 'o'], ['x', 'x', 'o'], ['o', 'x', 'x']] best=(0, 1)
state={(0, 1): 1000}
winner is o

I just won!
board=[[' ', 'x', 'x'], ['x', ' ', ' '], ['o'